# Assignment 1


In [1]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

Using TensorFlow backend.
c:\users\robin\anaconda\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [3]:
#create co-occurrence matrix
import pandas as pd
from sklearn import preprocessing

#dataset with unique words as index
words = list(tokenizer.word_index.keys())
matrix = pd.DataFrame(words, columns=['words'])
matrix.set_index('words', inplace=True)
columns = pd.DataFrame(columns=words)
matrix = pd.concat([matrix,columns])
matrix = matrix.reindex(columns.columns, axis=1)
matrix.fillna(0, inplace=True)

#inverse index to get word by code
inverse_index = dict((v,k) for k, v in tokenizer.word_index.items())

#compute score for every word-word couple
for line in corpus:
    for i, word_code in enumerate(line):
        word = inverse_index.get(word_code)
        for j in range(max(0, i-window_size_corpus), min(len(line), i+window_size_corpus+1)):
            if word_code != line[j]:
                matrix[word][inverse_index.get(line[j])] += 1

#normalize each sample
normalized_values = preprocessing.normalize(matrix.values)
matrix = pd.DataFrame(normalized_values, index=matrix.index, columns=matrix.columns)

matrix.head()

,key,rapidly,finish,curious,rest,made,drop,roof,cheerfully,own,...,trial,currants,gloves,inclined,noise,our,ahem,d,box,though
key,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rapidly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
finish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
curious,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#find cosine similarity to Alice, Dinah and Rabbit

alice_vector = matrix['alice'].values.reshape(1, -1)
rabbit_vector = matrix['rabbit'].values.reshape(1, -1)
dinah_vector = matrix['dinah'].values.reshape(1, -1)

alice_vs_rabbit = cosine_similarity(alice_vector, rabbit_vector)
alice_vs_dinah = cosine_similarity(alice_vector, dinah_vector)
dinah_vs_rabbit = cosine_similarity(dinah_vector, rabbit_vector)

print("Cosine similarity between Alice and Rabbit: " + str(alice_vs_rabbit[0][0]))
print("Cosine similarity between Alice and Dinah: " + str(alice_vs_dinah[0][0]))
print("Cosine similarity between Dinah and Rabbit: " + str(dinah_vs_rabbit[0][0]))

Cosine similarity between Alice and Rabbit: 0.05461780522416199
Cosine similarity between Alice and Dinah: 0.05073318700003422
Cosine similarity between Dinah and Rabbit: 0.034243448025346485


In [5]:
#find the closest words to Alice (nearest neighbors)

neigh = nn()
neigh.fit(matrix.values)

neighbors = neigh.kneighbors(alice_vector, 6)

for i, n in enumerate(neighbors[1][0]):
    if matrix.index[n] != 'alice':
        print("Word: {}\nDistance: {}\n".format(matrix.index[n], neighbors[0][0][i]))

Word: i
Distance: 3.6581715798218197

Word: she
Distance: 3.6627351402754176

Word: a
Distance: 3.664105104961175

Word: very
Distance: 3.665855124516278

Word: it
Distance: 3.6687653879055215



The five closest words to "Alice" appear to be: "I", "she", "a", "very" and "it". It makes sense that the personal pronouns "she" and "i" are the closest ones to "Alice", since they are used in very similiar contexts and they are probably often followed by the same verbs. E.g. "Alice sees the rabbit", "she sees the rabbit" or "I see the rabbit", in direct dialogs. As for the other three words, they do not bring valuable information, as they are all quite common in the english language. A possible way to avoid this would be to remove stopwords from the corpus, in order to focus only on more meaningful terms.

Discussion of the drawbacks:
- **Sparse matrix:** As can be seen in the output of the matrix, the word pair 'particular' and 'suddenly' never appear close to each other in the corpus, but there is still a cell reserved in the matrix for that pair. The same goes for many other pairs in the matrix, resulting in many cells with value 0. Obviously, this is not very efficient for storage.
- **Large matrix:** As can be seen in the word co-occurrence matrix for Alice In Wonderland that was created above, the matrix size is 1182 rows x 1182 columns, where 1182 is the number of tokens in the corpus. It is easy to imagine that this will become a problem for a bigger corpus, or for a corpus which has dynamic content, e.g. a web search engine.
- **Non-discriminative results:** As shown in the nearest neighbors calculation above, the words 'a', 'it', and 'very' do not give very meaningful correlation. However, they are among the most common words used in English language, so it makes sense that they appear often near the term 'Alice' (or any other terms in the corpus). Therefore, stopwords filter or even a different weighting score other than word occurence count should be used in order to find words correlation that is more meaningful.

In [6]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = matrix.values
for i, word in enumerate(tokenizer.word_index.keys()): 
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [7]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [8]:
#function definitions for CBOW

#generate data for CBOW
def generate_data_cbow(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            out_word = word
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    #out_words.append([word])
                    labels.append(np_utils.to_categorical(words[i], V))
            if labels != []:
                labels_sum = np.sum(labels, axis=0)
                context = labels_sum / np.linalg.norm(labels_sum)
                all_in.append(np.array([context],dtype=np.float32))
                all_out.append(np.array([[out_word]],dtype=np.int32))
    return (all_in,all_out)

#load the preprocessed CBOW data
def generate_data_cbow_from_file():
    f = open('data_cbow.txt' ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=float, sep=' ')
        inputs = np.asarray([inputs])
        outputs = np.fromstring(outputs, dtype=int, sep=' ')
        outputs = np.asarray([outputs])
        yield (inputs,outputs)

In [9]:
#prepare data for cbow
   
#get x and y's for data
x,y = generate_data_cbow(corpus,window_size,V)

#save the preprocessed data of CBOW
f = open('data_cbow.txt' ,'w')

for input,outcome  in zip(x,y):
    input = np.concatenate(input)
    f.write(" ".join(map(str, list(input))))
    f.write(",")
    outcome = np.concatenate(outcome)
    f.write(" ".join(map(str,list(outcome))))
    f.write("\n")
f.close()

#create CBOW model (TO DO)

for dim in [50, 150, 300]:
    #create model
    cbow = Sequential()
    cbow.add(Dense(input_dim=V, units=dim, kernel_initializer='uniform', activation='softmax'))
    #cbow.add(Reshape((dim, )))
    cbow.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
    
    #define loss function for CBOW
    cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')
    
    #train CBOW model
    print("\nTraining cbow for dim="+str(dim))
    for ite in range(5):
        loss = 0.
        for x, y in generate_data_cbow_from_file():
            loss += cbow.train_on_batch(x, y)
        print(ite, loss)
    
    #save vector representation to file
    f = open('vectors_cbow_'+str(dim)+'.txt' ,'w')
    f.write(" ".join([str(V-1),str(dim)]))
    f.write("\n")

    vectors = cbow.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        f.write(word)
        f.write(" ")
        f.write(" ".join(map(str, list(vectors[i,:]))))
        f.write("\n")
    f.close()

In [10]:
#define loss function


In [11]:
#train model


In [12]:
#function definitions for Skipgram

#generate data for Skipgram
def generate_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
                    
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    in_words.append([word])
                    labels.append(words[i])
            if in_words != []:
                all_in.append(np.array(in_words,dtype=np.int32))
                all_out.append(np_utils.to_categorical(labels, V))
    return (all_in,all_out)

#load the preprocessed Skipgram data
def generate_data_skipgram_from_file():
    f = open('data_skipgram.txt' ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=int, sep=' ')
        inputs = np.asarray(np.split(inputs, len(inputs)))
        outputs = np.fromstring(outputs, dtype=float, sep=' ')
        outputs = np.asarray(np.split(outputs, len(inputs)))
        yield (inputs,outputs)

In [13]:
#prepare data for Skipgram
   
#get x and y's for data
x,y = generate_data_skipgram(corpus,window_size,V)

#save the preprocessed data of Skipgram
f = open('data_skipgram.txt' ,'w')

for input,outcome  in zip(x,y):
    input = np.concatenate(input)
    f.write(" ".join(map(str, list(input))))
    f.write(",")
    outcome = np.concatenate(outcome)
    f.write(" ".join(map(str,list(outcome))))
    f.write("\n")
f.close()

In [14]:
#create Skipgram model

for dim in [50, 150, 300]:
    #create model
    skipgram = Sequential()
    skipgram.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
    skipgram.add(Reshape((dim, )))
    skipgram.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
    
    #define loss function for Skipgram
    skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta')
    
    #train skipgram model
    print("\nTraining skipgram for dim="+str(dim))
    for ite in range(10):
        loss = 0.
        for x, y in generate_data_skipgram_from_file():
            loss += skipgram.train_on_batch(x, y)
        print(ite, loss)
    
    #save vector representation to file
    f = open('vectors_skipgram_'+str(dim)+'.txt' ,'w')
    f.write(" ".join([str(V-1),str(dim)]))
    f.write("\n")

    vectors = skipgram.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        f.write(word)
        f.write(" ")
        f.write(" ".join(map(str, list(vectors[i,:]))))
        f.write("\n")
    f.close()


Training skipgram for dim=50
0 41271.354187488556
1 39126.8399810791
2 39288.232560157776
3 39363.88622379303
4 39432.414731025696
5 39512.000215768814
6 39608.41395294666
7 39723.93453395367
8 39852.47196030617
9 39987.056107997894

Training skipgram for dim=150
0 41222.655864715576
1 38923.38359093666
2 38988.32388424873
3 39005.66209816933
4 39038.837849497795
5 39093.74402093887
6 39156.200922489166
7 39216.76563632488
8 39267.67642009258
9 39305.3429261446

Training skipgram for dim=300
0 41159.68825006485
1 38730.84347319603
2 38713.5351524353
3 38676.76848196983
4 38648.0355219841
5 38631.884105205536
6 38611.777688622475
7 38578.92392885685
8 38533.39706158638
9 38484.56349503994


In [15]:
#create CBOW model with additional dense layer


In [16]:
#define loss function for CBOW + dense


In [17]:
#train model for CBOW + dense


In [18]:
#create Skipgram model with additional dense layer

for dim in [50, 150, 300]:
    #create model
    skipgram = Sequential()
    skipgram.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
    skipgram.add(Reshape((dim, )))
    skipgram.add(Dense(128, activation="relu"))
    skipgram.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))
    
    #define loss function for Skipgram + dense
    skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta')
    
    #train model for Skipgram + dense
    print("\nTraining skipgram with dense layer for dim="+str(dim))
    for ite in range(10):
        loss = 0.
        for x, y in generate_data_skipgram_from_file():
            loss += skipgram.train_on_batch(x, y)
        print(ite, loss)
    
    #save vector representation to file
    f = open('vectors_skipgram_dense_'+str(dim)+'.txt' ,'w')
    f.write(" ".join([str(V-1),str(dim)]))
    f.write("\n")

    vectors = skipgram.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        f.write(word)
        f.write(" ")
        f.write(" ".join(map(str, list(vectors[i,:]))))
        f.write("\n")
    f.close()


Training skipgram with dense layer for dim=50
0 39398.1423869133
1 38929.386723041534
2 38784.686121463776
3 38609.711948275566
4 38497.59895861149
5 38394.93743932247
6 38308.49504172802
7 38234.45247685909
8 38150.21204769611
9 38082.67344665527

Training skipgram with dense layer for dim=150
0 39297.98832154274
1 38703.91829586029
2 38481.45571088791
3 38229.65763485432
4 38031.8431353569
5 37858.61486911774
6 37678.68290746212
7 37499.43464541435
8 37309.12741732597
9 37134.485713362694

Training skipgram with dense layer for dim=300
0 39210.35425782204
1 38579.465814352036
2 38347.45797133446
3 38155.709585785866
4 37954.37697124481
5 37795.83253669739
6 37605.89838516712
7 37395.25325042009
8 37157.994569182396
9 36893.878751158714


In [19]:
#load vectors from files
cbows = []
cbows_dense = []
skipgrams = []
skipgrams_dense = []
    
for dim in [50, 150, 300]:
    f = open('vectors_skipgram_'+str(dim)+'.txt' ,'r')
    d = {}
    for row in f.readlines()[1:]:
        line = row.split()
        d[line[0]] = line[1:]
    skipgrams.append(d)
    
    f = open('vectors_skipgram_dense_'+str(dim)+'.txt' ,'r')
    d = {}
    for row in f.readlines()[1:]:
        line = row.split()
        d[line[0]] = line[1:]
    skipgrams_dense.append(d)
    
#save as separate dictionaries
skipgram_vectors_50 = skipgrams[0]
skipgram_vectors_150 = skipgrams[1]
skipgram_vectors_300 = skipgrams[2]
skipgram_dense_vectors_50 = skipgrams_dense[0]
skipgram_dense_vectors_150 = skipgrams_dense[1]
skipgram_dense_vectors_300 = skipgrams_dense[2]

In [79]:
#Implement your own analogy function
def analogy_function(dictionary):
    scores = []
    f = open('analogy_alice.txt' ,'r')
    for row in f:
        line = row.split()
        if all(x in dictionary for x in line):
            A = np.array(dictionary[line[0]], dtype=float)
            B = np.array(dictionary[line[1]], dtype=float)
            C = np.array(dictionary[line[2]], dtype=float)
            D = np.array(dictionary[line[3]], dtype=float)        
            scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])
            
    return sum(scores)/len(scores)


f = open('analogy_alice.txt' ,'r')
skipgram_50_scores = []
skipgram_150_scores = []
skipgram_300_scores = []
skipgram_dense_50_scores = []
skipgram_dense_150_scores = []
skipgram_dense_300_scores = []

for row in f:
    line = row.split()
    if all(x in skipgram_vectors_50 for x in line):
        
        #skipgram 50
        A = np.array(skipgram_vectors_50[line[0]], dtype=float)
        B = np.array(skipgram_vectors_50[line[1]], dtype=float)
        C = np.array(skipgram_vectors_50[line[2]], dtype=float)
        D = np.array(skipgram_vectors_50[line[3]], dtype=float)        
        skipgram_50_scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])
        
        #skipgram 150
        A = np.array(skipgram_vectors_150[line[0]], dtype=float)
        B = np.array(skipgram_vectors_150[line[1]], dtype=float)
        C = np.array(skipgram_vectors_150[line[2]], dtype=float)
        D = np.array(skipgram_vectors_150[line[3]], dtype=float)        
        skipgram_150_scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])
        
        #skipgram 300
        A = np.array(skipgram_vectors_300[line[0]], dtype=float)
        B = np.array(skipgram_vectors_300[line[1]], dtype=float)
        C = np.array(skipgram_vectors_300[line[2]], dtype=float)
        D = np.array(skipgram_vectors_300[line[3]], dtype=float)        
        skipgram_300_scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])
        
        #skipgram + dense 50
        A = np.array(skipgram_dense_vectors_50[line[0]], dtype=float)
        B = np.array(skipgram_dense_vectors_50[line[1]], dtype=float)
        C = np.array(skipgram_dense_vectors_50[line[2]], dtype=float)
        D = np.array(skipgram_dense_vectors_50[line[3]], dtype=float)        
        skipgram_dense_50_scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])
        
        #skipgram + dense 150
        A = np.array(skipgram_dense_vectors_150[line[0]], dtype=float)
        B = np.array(skipgram_dense_vectors_150[line[1]], dtype=float)
        C = np.array(skipgram_dense_vectors_150[line[2]], dtype=float)
        D = np.array(skipgram_dense_vectors_150[line[3]], dtype=float)        
        skipgram_dense_150_scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])
        
        #skipgram + dense 300
        A = np.array(skipgram_dense_vectors_300[line[0]], dtype=float)
        B = np.array(skipgram_dense_vectors_300[line[1]], dtype=float)
        C = np.array(skipgram_dense_vectors_300[line[2]], dtype=float)
        D = np.array(skipgram_dense_vectors_300[line[3]], dtype=float)        
        skipgram_dense_300_scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])

        
        
#skipgram score
skipgram_50_score = analogy_function(skipgram_vectors_50)
print("Analogy score for Skipgram model with dim=50: " + str(skipgram_50_score))
skipgram_150_score = analogy_function(skipgram_vectors_510)
print("Analogy score for Skipgram model with dim=150: " + str(skipgram_150_score))
skipgram_50_score = analogy_function(skipgram_vectors_50)
print("Analogy score for Skipgram model with dim=50: " + str(skipgram_50_score))
print()

#skipgram + dense scores
skipgram_dense_50_avg = sum(skipgram_dense_50_scores)/len(skipgram_dense_50_scores)
print("Analogy score for Skipgram + dense model with dim=50: " + str(skipgram_dense_50_avg))
skipgram_dense_150_avg = sum(skipgram_dense_150_scores)/len(skipgram_dense_150_scores)
print("Analogy score for Skipgram + dense model with dim=150: " + str(skipgram_150_avg))
skipgram_dense_300_avg = sum(skipgram_dense_300_scores)/len(skipgram_dense_300_scores)
print("Analogy score for Skipgram + dense model with dim=300: " + str(skipgram_dense_300_avg))
print()

Analogy score for Skipgram model with dim=50: 0.1538180261626648
Analogy score for Skipgram model with dim=150: 0.09847292209927096
Analogy score for Skipgram model with dim=300: 0.07711539410906129

Analogy score for Skipgram + dense model with dim=50: -0.0020859167319388513
Analogy score for Skipgram + dense model with dim=150: 0.09847292209927096
Analogy score for Skipgram + dense model with dim=300: -0.0003669102820264841



Comparison performance:

In [78]:
#Comparison to the co-occurrance matrix embeddings
f = open('analogy_alice.txt' ,'r')
matrix_scores = []

for row in f:
    line = row.split()
    if all(x in matrix for x in line):
        A = np.array(matrix[line[0]].values, dtype=float)
        B = np.array(matrix[line[1]].values, dtype=float)
        C = np.array(matrix[line[2]].values, dtype=float)
        D = np.array(matrix[line[3]].values, dtype=float)
        matrix_scores.append(cosine_similarity((A - B + C).reshape(1, -1),D.reshape(1, -1))[0][0])
    
#matrix scores
matrix_avg = sum(matrix_scores)/len(matrix_scores)
print("Analogy score for co-occurrance matrix: " + str(matrix_avg))

Analogy score for co-occurrance matrix: -0.0032230167361160847


In [21]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [22]:
#load pretrained word embeddings of word2vec

path_word2vec = "/GoogleNews-vectors-negative300.bin"

#word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [23]:
#load pretraind word embeddings of Glove

path = "/glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

#glove = KeyedVectors.load_word2vec_format(path, binary=False)

NameError: name 'gensim' is not defined

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings